In [1]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

from models.cnn import mini_XCEPTION
from utils.datasets import DataManager
from utils.datasets import split_data
from utils.preprocessor import preprocess_input

# parameters
batch_size = 32
num_epochs = 30
input_shape = (64, 64, 1)
validation_split = .2
verbose = 1
num_classes = 7
patience = 50
base_path = '../trained_models/emotion_models/'

# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

# model parameters/compilation
model = mini_XCEPTION(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


datasets = ['fer2013']
for dataset_name in datasets:
    print('Training dataset:', dataset_name)

    # callbacks
    log_file_path = base_path + dataset_name + '_emotion_training.log'
    csv_logger = CSVLogger(log_file_path, append=False)
    early_stop = EarlyStopping('val_loss', patience=patience)
    reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
    trained_models_path = base_path + dataset_name + '_mini_XCEPTION'
    model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
    model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
    callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

    # loading dataset
    data_loader = DataManager(dataset_name, image_size=input_shape[:2])
    faces, emotions = data_loader.get_data()
    faces = preprocess_input(faces)
    num_samples, num_classes = emotions.shape
    train_data, val_data = split_data(faces, emotions, validation_split)
    train_faces, train_emotions = train_data
    model.fit_generator(data_generator.flow(train_faces, train_emotions,
                                            batch_size),
                        steps_per_epoch=len(train_faces) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=val_data)


Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 64, 64, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 62, 62, 8)     72          input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 62, 62, 8)     32          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 62, 62, 8)     0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

Epoch 1/30
898/897 [==============================] - 871s - loss: 1.7837 - acc: 0.3209 - val_loss: 1.5769 - val_acc: 0.4093
Epoch 2/30
898/897 [==============================] - 819s - loss: 1.4995 - acc: 0.4405 - val_loss: 1.4788 - val_acc: 0.4483
Epoch 3/30
898/897 [==============================] - 852s - loss: 1.3764 - acc: 0.4876 - val_loss: 1.4382 - val_acc: 0.4540
Epoch 4/30
898/897 [==============================] - 829s - loss: 1.3123 - acc: 0.5105 - val_loss: 1.4183 - val_acc: 0.4967
Epoch 5/30
898/897 [==============================] - 818s - loss: 1.2613 - acc: 0.5291 - val_loss: 1.3011 - val_acc: 0.5134
Epoch 6/30
898/897 [==============================] - 814s - loss: 1.2225 - acc: 0.5411 - val_loss: 1.1876 - val_acc: 0.5521
Epoch 7/30
898/897 [==============================] - 814s - loss: 1.2087 - acc: 0.5474 - val_loss: 1.2170 - val_acc: 0.5474
Epoch 8/30
898/897 [==============================] - 825s - loss: 1.1771 - acc: 0.5603 - val_loss: 1.2853 - val_acc: 0.5171
